<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
<center>Автор материала: программист-исследователь Mail.Ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ [Юрий Кашницкий](https://yorko.github.io/)

# <center> Capstone проект №1 <br> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 5.  Соревнование Kaggle "Catch Me If You Can"

На этой неделе мы вспомним про концепцию стохастического градиентного спуска и опробуем классификатор Scikit-learn SGDClassifier, который работает намного быстрее на больших выборках, чем алгоритмы, которые мы тестировали на 4 неделе. Также мы познакомимся с данными [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) Kaggle по идентификации пользователей и сделаем в нем первые посылки. По итогам этой недели дополнительные баллы получат те, кто попадет в топ-30 публичного лидерборда соревнования.

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
**Также рекомендуется вернуться и просмотреть [задание](https://www.coursera.org/learn/supervised-learning/programming/t2Idc/linieinaia-rieghriessiia-i-stokhastichieskii-ghradiientnyi-spusk) "Линейная регрессия и стохастический градиентный спуск" 1 недели 2 курса специализации.**

### Задание
1. Заполните код в этой тетрадке 
2. Если вы проходите специализацию Яндеса и МФТИ, пошлите тетрадку в соответствующем Peer Review. <br> Если вы проходите курс ODS, выберите ответы в [веб-форме](https://docs.google.com/forms/d/1pLsegkAICL9PzOLyAeH9DmDOBfktte0l8JW75uWcTng). 

In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from tqdm import tqdm_notebook
import re

**Считаем данные [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [53]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = r'C:\Users\Дмитрий Мошкин\Documents\GitHub\moshkin\MFTI_Yandex_DA_specialisation\6 course\catch-me-if-you-can'

In [55]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')

In [56]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [57]:
train_test_df = pd.concat([train_df, test_df])

In [35]:
train_test_df.columns

Index(['site1', 'site10', 'site2', 'site3', 'site4', 'site5', 'site6', 'site7',
       'site8', 'site9', 'target', 'time1', 'time10', 'time2', 'time3',
       'time4', 'time5', 'time6', 'time7', 'time8', 'time9'],
      dtype='object')

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 21 columns):
site1     253561 non-null int64
time1     253561 non-null object
site2     250098 non-null float64
time2     250098 non-null object
site3     246919 non-null float64
time3     246919 non-null object
site4     244321 non-null float64
time4     244321 non-null object
site5     241829 non-null float64
time5     241829 non-null object
site6     239495 non-null float64
time6     239495 non-null object
site7     237297 non-null float64
time7     237297 non-null object
site8     235224 non-null float64
time8     235224 non-null object
site9     233084 non-null float64
time9     233084 non-null object
site10    231052 non-null float64
time10    231052 non-null object
target    253561 non-null int64
dtypes: float64(9), int64(2), object(10)
memory usage: 42.6+ MB


In [7]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 20 columns):
site1     82797 non-null int64
time1     82797 non-null object
site2     81308 non-null float64
time2     81308 non-null object
site3     80075 non-null float64
time3     80075 non-null object
site4     79182 non-null float64
time4     79182 non-null object
site5     78341 non-null float64
time5     78341 non-null object
site6     77566 non-null float64
time6     77566 non-null object
site7     76840 non-null float64
time7     76840 non-null object
site8     76151 non-null float64
time8     76151 non-null object
site9     75484 non-null float64
time9     75484 non-null object
site10    74806 non-null float64
time10    74806 non-null object
dtypes: float64(9), int64(1), object(10)
memory usage: 13.3+ MB


**В обучающей выборке – 2297 сессий одного пользователя (Alice) и 251264 сессий – других пользователей, не Элис. Дисбаланс классов очень сильный, и смотреть на долю верных ответов (accuracy) непоказательно.**

**Пока для прогноза будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [36]:
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')

In [37]:
train_test_df_sites.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518
3,14769,39,14768,14769,37,39,14768,14768,14768,14768
4,782,782,782,782,782,782,782,782,782,782
5,22,177,175,178,177,178,175,177,177,178
6,570,21,570,21,21,0,0,0,0,0
7,803,23,5956,17513,37,21,803,17514,17514,17514
8,22,21,29,5041,14422,23,21,5041,14421,14421
9,668,940,942,941,941,942,940,23,21,22


**Создайте разреженные матрицы *X_train_sparse* и *X_test_sparse* аналогично тому, как мы это делали ранее. Используйте объединенную матрицу *train_test_df_sites*, потом разделите обратно на обучающую и тестовую части.**

Обратите внимание на то, что в  сессиях меньше 10 сайтов  у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.

**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

In [38]:
def sparsecreator(X):
    '''takes matrix and returns sparse matrix alternative'''
    
    indptr = [0]
    indices = []
    vocabulary = {}
    data = []
    for row in X:
        for column in row:
            index = vocabulary.setdefault(column, column)
            indices.append(index)
            data.append(1)
        indptr.append(len(indices))
    return csr_matrix((data, indices, indptr), dtype=int)[:, 1:] # deleting 0 index

In [39]:
train_test_sparse = sparsecreator(train_test_df_sites.values)
train_test_sparse = train_test_sparse[:, 1:]
X_train_sparse = train_test_sparse[:253561]
X_test_sparse = train_test_sparse[253561:]
y = train_df.target.values

**<font color='red'>Вопрос 1. </font> Выведите размерности матриц *X_train_sparse* и *X_test_sparse* – 4 числа на одной строке через пробел: число строк и столбцов матрицы *X_train_sparse*, затем число строк и столбцов матрицы *X_test_sparse*.**

In [40]:
for i in X_train_sparse.shape + X_test_sparse.shape:
    print(str(i), end=' ')

253561 48370 82797 48370 

**Сохраним в pickle-файлы объекты *X_train_sparse*, *X_test_sparse* и *y* (последний – в файл *kaggle_data/train_target.pkl*).**

In [12]:
with open(os.path.join(PATH_TO_DATA, 'X_train_sparse.pkl'), 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_test_sparse.pkl'), 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'train_target.pkl'), 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl, protocol=2)

**Разобьем обучающую выборку на 2 части в пропорции 7/3, причем не перемешивая. Исходные данные упорядочены по времени, тестовая выборка по времени четко отделена от обучающей, это же соблюдем и здесь.**

In [41]:
train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

**Создайте объект `sklearn.linear_model.SGDClassifier` с логистической функцией потерь и параметром *random_state*=17. Остальные параметры оставьте по умолчанию, разве что *n_jobs*=-1 никогда не помешает. Обучите  модель на выборке `(X_train, y_train)`.**

In [42]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=17, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

**Сделайте прогноз в виде предсказанных вероятностей того, что это сессия Элис, на отложенной выборке *(X_valid, y_valid)*.**

In [43]:
logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)

**<font color='red'>Вопрос 2. </font> Посчитайте ROC AUC логистической регрессии, обученной с помощью стохастического градиентного спуска, на отложенной выборке. Округлите до 3 знаков после разделителя.**

In [44]:
round(roc_auc_score(y_valid, logit_valid_pred_proba[:, 1]), 3)

0.934

**Сделайте прогноз в виде предсказанных вероятностей отнесения к классу 1 для тестовой выборки с помощью той же *sgd_logit*, обученной уже на всей обучающей выборке (а не на 70%).**

In [45]:
%%time
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train_sparse, y)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_sparse)

Wall time: 285 ms


**Запишите ответы в файл и сделайте посылку на Kaggle. Дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] Coursera_Username", чтоб можно было легко идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard/public).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить.**

In [46]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [47]:
write_to_submission_file(logit_test_pred_proba[:, 1], 'w5_a1.csv')

## Обучение логрегрессии на выборке с дополнительными признаками

### Обучение на уменьшенной выборке

Для начала проведем исследование на выборке 150 юзеров, включая Алису. Я переименовал Алису в user77777777.csv

Подготовим словарь с индексами, для этого используем функцию, подготовленную на второй неделе.

In [7]:
PATH_TO_DATA = r'C:\Users\Дмитрий Мошкин\Documents\GitHub\moshkin\MFTI_Yandex_DA_specialisation\6 course\catch-me-if-you-can'

In [11]:
def func1(data, filename, session_length=10):
    """takes user history table and returns array with sessions history where number of 
    columns == session_length+1 (the last column is user's index)
    fills empty values in last session (if there is) with 0
    """
    
    i = True

    while i:
        try:
            data = data.reshape((-1, session_length))
            i = False
        except:
            data = np.append(data, (0))
     
    # Add column with user index (index is pulled from filename):
    
    userindex = re.findall(r'(\d*).csv', filename)[0]
    userindexcolumn = np.array([userindex]*len(data))[:, np.newaxis]
    data = np.hstack((data, userindexcolumn))
    
    return data
#pd.DataFrame(func1(path_to_file)).head()

def func2(path_to_csv_files):
    
    '''Parce whole files' catalog. Applies func1 to all files.
    Returns Numpy array with sessions, dictionary with websites appearence frequency
    '''
    
    dic = {}
    data = None
    
    os.chdir(path_to_csv_files)
    filenames = sorted(os.listdir())
    
    for filename in tqdm_notebook(filenames):
        if '.csv' in filename:
            usersiteslist = pd.read_csv(os.path.join(path_to_csv_files, filename)).site.values
            counts = np.unique(usersiteslist, return_counts=True)
        
        # Counting websites appearances:
        
            for i, j in zip(counts[0], counts[1]):
                try:
                    dic[i] += j
                except KeyError:
                    dic[i] = j
                
        # Filling the massive with sessions:
        
            try:
                data = np.append(data, func1(usersiteslist, filename), axis=0)
            
            except ValueError:
                data = func1(usersiteslist, filename)
            
    return data, dic

def func3(data, dic):
    '''
    Sorts websites' names by frequency and attach indeces in order where 1 is the most frequent 
    website. Replaces websites' names with theirs indeces.
    Returns: dict {'site_string': [site_id, site_freq]}; numpy array with user-connected sessions
    '''

    # Create list representation of dict and sort websites by appearing frequency:

    counts = [(key, value) for key, value in dic.items()]
    counts = sorted(counts, key = lambda x: x[1], reverse=True)

    # Create dict

    dic = {}
    i = 1
    for key, value in counts:
        dic[key] = [i, value]
        i += 1
        
    for i in range(data.shape[0]):
        for j in range(data[:, 0:-1].shape[1]):
            if data[i][j] != 0:
                data[i][j] = dic[data[i][j]][0]
        
    return data, dic

def prepare_train_set(path_to_csv_files, session_length=10):
    '''wrapper above func1, func2, func3.
    Takes path to .csv tables and session lenght (10 by default)
    Returns: DataFrame with user-connected indexed sessions; 
             dict {'site_string': [site_id, site_freq]}
    '''
    
    data, dic = func2(path_to_csv_files)
    data, dic = func3(data, dic)
    columns = list(map(lambda x: 'site'+str(x), range(1, data.shape[1]))) + ['user_id']
    data = pd.DataFrame(data, columns=columns)
    data.index.name = 'session_id'
    return data, dic

In [9]:
_, dic150 = prepare_train_set(os.path.join(PATH_TO_DATA, 'train150'))

In [109]:
with open(os.path.join(PATH_TO_DATA, 'site_freq_150.pkl'), 'wb') as f:
    pickle.dump(dic150, f, protocol=2)

Подготовим выборку из 150 пользователей с сессией 15 и окном 5, так как именно при таких параметрах наблюдались лучшие результаты при решении задачи классификации на SVM. Будем применять бинарную классификацию, поэтому Алисе будет присвоена метка '1', всем остальным - '0'.

In [110]:
def table_trasformer(data, filename, dic, additional_feature_number, session_length=10, window_size=10):
    '''transform single table'''
    
#   Pull websites in original order and replace names with indices from dic:
    
    sites = [dic[i][0] for i in data.site.values]
    if len(data) < session_length:
        unique_sites_feature = len(np.unique(sites))
        sites = sites + [0]*abs(len(sites) - session_length)
    
    i = 0
    while True:
        
#   Create session-row:
        
        row = sites[i:(i + session_length)]
        if 0 not in row:
            unique_sites_feature = len(np.unique(row))
        row = row + [0]*(additional_feature_number) # [0] in row for each additional feature
        if sum(row) == 0:
            break
        if i == 0:
            frame = np.array([row])
        else:
            try:
                frame = np.append(frame, [row], axis=0)
            except ValueError:
                delta = abs(len(frame[0]) - len(row))
                frame = np.append(frame, [row + [0]*delta], axis=0)

#   Add features:
        
        
        time_stamps = data.timestamp[i:(i + session_length)].apply(pd.to_datetime)    
        time_diffs = time_stamps.diff()
        ind = 0    
        for gap in time_diffs[1:]:
            frame[-1][session_length+ind] = gap.seconds
            ind += 1
            
            
        session_timespan_feature = (time_stamps.max() - time_stamps.min()).seconds
        day_of_week_feature = time_stamps.min().dayofweek
        start_hour_feature = time_stamps.min().hour
        userindex = re.findall(r'(\d*).csv', filename)[0]
        frame[-1][-5] = session_timespan_feature
        frame[-1][-4] = unique_sites_feature
        frame[-1][-3] = start_hour_feature
        frame[-1][-2] = day_of_week_feature
        if userindex == '77777777':
            frame[-1][-1] = 1Подготовим выборку из 150 пользователей с сессией 15 и окном 5, так как именно при таких параметрах наблюдались лучшие результаты при решении задачи классификации на SVM
        else:
            frame[-1][-1] = 0
        
        i += window_size

    return frame


def prepare_train_set_with_fe(
            path_to_csv_files, site_freq_path, 
            feature_names = ['site' + str(i) for i in range(1,11)] + \
            ['time_diff' + str(j) for j in range(1,10)] + \
            ['session_timespan', '#unique_sites', 'start_hour', 'day_of_week', 'target'],
            session_length=10, window_size=10):
    '''
    takes:
        path_to_csv_files - path to CSV files
        site_freq_path - path to a pickle file containing frequent dict
        feature_names - dict with names of features
        session_length - session legnth, int
        window_size - window size, int
        
    returns:
        matrix with session_length rows and len(feature_names) columns
     '''
    
    X = None
    additional_feature_number = len(feature_names) - session_length
# Pull filenames to the list:

    os.chdir(path_to_csv_files)
    filenames = sorted(os.listdir())

# Load dictionary with websites' indices:
    
    with open(site_freq_path, 'rb') as f:
        dic = pickle.load(f)
        
# Retrive website name from a file, transform and merge into overall array:
        
    for filename in tqdm_notebook(filenames):
        if '.csv' in filename:
            file = pd.read_csv(os.path.join(path_to_csv_files, filename))
            csvresult = table_trasformer(file, filename, dic, additional_feature_number, session_length, 
                              window_size)
            try:
                X = np.append(X, csvresult, axis=0)
            except ValueError:
                X = csvresult
    return pd.DataFrame(X, columns=feature_names)
#     return X

In [111]:
feature_names = ['site' + str(i) for i in range(1,16)] + \
            ['time_diff' + str(j) for j in range(1,15)] + \
            ['session_timespan', '#unique_sites', 'start_hour', 'day_of_week', 'target']
path_to_csv_files = os.path.join(PATH_TO_DATA, 'train150')
site_freq_path = os.path.join(PATH_TO_DATA, 'site_freq_150.pkl')

X = prepare_train_set_with_fe(path_to_csv_files, site_freq_path, 
                              feature_names, session_length=15, window_size=5)

Создадим разряженную матрицу и вектор с метками классов:

In [134]:
X_sparse = sparsecreator(X.iloc[:, :-1].values)

# Удалим столбец с индексом сайта '0', так как он имеет природу, не относящуюся на самом деле к какому-либо сайту:

X_sparse = X_sparse[:, 1:] 
y = X.iloc[:, -1] 

Разобьем получившуюся выборку на обучение и тест с пропорцией на тест 0.3. Выполним стратификацию, что бы сессии Алисы были в равных пропорциях представлены на обучении и тесте.

In [144]:
X_train, X_valid, y_train, y_valid = train_test_split(X_sparse, y, test_size=0.3, stratify=y, random_state=17)

Обучим логистическую регрессию с применением SGDC на тестовой выборке и посмотрим на ROCAUC на отложенной.

In [148]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train, y_train)
logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)
round(roc_auc_score(y_valid, logit_valid_pred_proba[:, 1]), 3)

0.964

Результат выглядит оптимистично. Попробуем обучить данные целиком.

### Обучение на полной выборке

Подготовим обучающую выборку.

In [157]:
'user0002.csv'[-8:-4]

'0002'

In [59]:
train_test_df.head(2)

,site1,site10,site2,site3,site4,site5,site6,site7,site8,site9,...,time1,time10,time2,time3,time4,time5,time6,time7,time8,time9
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,890,1518.0,941.0,3847.0,941.0,942.0,3846.0,3847.0,3846.0,1516.0,...,2014-02-22 11:19:50,2014-02-22 11:20:16,2014-02-22 11:19:50,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:52,2014-02-22 11:19:52,2014-02-22 11:20:15


In [ ]:
def prepare_train_set_with_fe(path_to_csv_files, site_freq_path, feature_names,
                                    session_length=10, window_size=10):

    site_freqs_dict = pd.read_pickle(site_freq_path)
    site_ids_dict = {key: val[0] for key, val in site_freqs_dict.items()}
    rows_list = []
    target = []

    for filepath in tqdm_notebook(glob(os.path.join(path_to_csv_files, '*.csv'))):
        timestamps = []
        sites = []
        userid = int(filepath[-8:-4].lstrip('0')) 

        with open(filepath, 'r') as f:
            f.readline()
            for line in f:
                time, site = line.strip().split(',')
                timestamps.append(pd.Timestamp(time))  # format='%Y-%m-%d %H:%M:%S'
                sites.append(site)

        pos = 0
        while pos < len(sites):
            rows_list.append({'site' + str(key+1): 
                              val for key, val in enumerate(sites[pos:pos+session_length])}) 
            rows_list[-1].update({'time_diff' + str(key):  
                                  val for key, val in enumerate(timestamps[pos:pos+session_length])})
            if userid == 7777 or userid == '7777':
                target.append(1)
            else:
                target.append(0)
                
            pos += window_size
    

    sessions = pd.DataFrame(rows_list, columns=['site' + str(i+1) for i in range(session_length)]
                            + ['time_diff' + str(i) for i in range(session_length)])
    sessions.iloc[:,:session_length] = \
                    sessions.iloc[:,:session_length].apply(lambda x: x.map(site_ids_dict, na_action='ignore'))
    min_time = sessions.iloc[:,session_length:session_length*2].min(axis=1)
    timespan = (sessions.iloc[:,session_length:session_length*2].max(axis=1) - 
                min_time).dt.seconds
    for i in range(session_length*2-1, session_length, -1):
        sessions.iloc[:,i] = (sessions.iloc[:,i] - sessions.iloc[:,i-1]).dt.seconds
    sessions['session_timespan'] = timespan
    sessions['#unique_sites'] = sessions.iloc[:,:session_length].nunique(axis=1).astype(np.int8)  
                                # this is time consuming operation
    sessions['start_hour'] = min_time.dt.hour.astype(np.int8)
    sessions['day_of_week'] = min_time.dt.dayofweek.astype(np.int8)
    sessions['target'] = np.array(target).astype(np.int16)
    sessions.drop('time_diff0', axis=1, inplace=True)
    sessions.fillna(0, inplace = True)
    sessions.iloc[:, :session_length] = sessions.iloc[:, :session_length].astype(np.int16)
    sessions.iloc[:, session_length:-4] = sessions.iloc[:, session_length:-4].astype(np.int32)
    
    return sessions

In [18]:
def prepare_train_set_with_fe(path_to_csv_files, site_freq_path, feature_names,
                                    session_length=10, window_size=10):

    site_freqs_dict = pd.read_pickle(site_freq_path)
    site_ids_dict = {key: val[0] for key, val in site_freqs_dict.items()}
    rows_list = []
    target = []

    for filepath in tqdm_notebook(glob(os.path.join(path_to_csv_files, '*.csv'))):
        timestamps = []
        sites = []
        userid = int(filepath[-8:-4].lstrip('0')) 

        with open(filepath, 'r') as f:
            f.readline()
            for line in f:
                time, site = line.strip().split(',')
                timestamps.append(pd.Timestamp(time))  # format='%Y-%m-%d %H:%M:%S'
                sites.append(site)

        pos = 0
        while pos < len(sites):
            rows_list.append({'site' + str(key+1): 
                              val for key, val in enumerate(sites[pos:pos+session_length])}) 
            rows_list[-1].update({'time_diff' + str(key):  
                                  val for key, val in enumerate(timestamps[pos:pos+session_length])})
            if userid == 7777 or userid == '7777':
                target.append(1)
            else:
                target.append(0)
                
            pos += window_size
    

    sessions = pd.DataFrame(rows_list, columns=['site' + str(i+1) for i in range(session_length)]
                            + ['time_diff' + str(i) for i in range(session_length)])
    sessions.iloc[:,:session_length] = \
                    sessions.iloc[:,:session_length].apply(lambda x: x.map(site_ids_dict, na_action='ignore'))
    min_time = sessions.iloc[:,session_length:session_length*2].min(axis=1)
    timespan = (sessions.iloc[:,session_length:session_length*2].max(axis=1) - 
                min_time).dt.seconds
    for i in range(session_length*2-1, session_length, -1):
        sessions.iloc[:,i] = (sessions.iloc[:,i] - sessions.iloc[:,i-1]).dt.seconds
    sessions['session_timespan'] = timespan
    sessions['#unique_sites'] = sessions.iloc[:,:session_length].nunique(axis=1).astype(np.int8)  
                                # this is time consuming operation
    sessions['start_hour'] = min_time.dt.hour.astype(np.int8)
    sessions['day_of_week'] = min_time.dt.dayofweek.astype(np.int8)
    sessions['target'] = np.array(target).astype(np.int16)
    sessions.drop('time_diff0', axis=1, inplace=True)
    sessions.fillna(0, inplace = True)
    sessions.iloc[:, :session_length] = sessions.iloc[:, :session_length].astype(np.int16)
    sessions.iloc[:, session_length:-4] = sessions.iloc[:, session_length:-4].astype(np.int32)
    
    return sessions

In [20]:
# Словарь:

_, dic150 = prepare_train_set(os.path.join(PATH_TO_DATA, 'train\other_user_logs'))

with open(os.path.join(PATH_TO_DATA, 'site_freq_all.pkl'), 'wb') as f:
    pickle.dump(dic150, f, protocol=2)

# Подготовим выборку с сессией 15 и окном 5, для Алисы метка '1', для остальных '0'
    
feature_names = ['site' + str(i) for i in range(1,11)] + \
            ['time_diff' + str(j) for j in range(1,10)] + \
            ['session_timespan', '#unique_sites', 'start_hour', 'day_of_week', 'target']
path_to_csv_files = os.path.join(PATH_TO_DATA, 'train\other_user_logs')
site_freq_path = os.path.join(PATH_TO_DATA, 'site_freq_all.pkl')

X = prepare_train_set_with_fe(path_to_csv_files, site_freq_path, 
                              feature_names, session_length=10, window_size=10)

In [21]:
X.shape

(242007, 24)

In [29]:
test_df.shape[0] + train_df.shape[0]

336358

Создадим разряженную матрицу и вектор с метками классов:

In [168]:
X_sparse = sparsecreator(X.iloc[:, :-1].values)

# Удалим столбец с индексом сайта '0', так как он имеет природу, не относящуюся на самом деле к какому-либо сайту:

X_sparse = X_sparse[:, 1:] 
y = X.iloc[:, -1] 

In [170]:
with open(os.path.join(PATH_TO_DATA, 'X_sparse_allusers.pkl'), 'wb') as f:
    pickle.dump(X_sparse, f, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'y_allusers.pkl'), 'wb') as f:
    pickle.dump(y, f, protocol=2)   

Обучим логистическую регрессию с применением SGDC на обучающей выборке и выполним предсказание на тестовой выборке с kaggle.

In [186]:
X_sparse[400411:].shape

(82797, 86398)

In [61]:
X_test_sparse

<82797x48370 sparse matrix of type '<class 'numpy.int32'>'
	with 781799 stored elements in Compressed Sparse Row format>

In [91]:
train_test_df.iloc[2][11:13].apply(pd.to_datetime).min()

Timestamp('2013-12-16 16:40:17')

In [74]:
lst = list(map(pd.Timestamp, [train_df.iloc[2][1], train_df.iloc[2][3]]))

In [79]:
min(list)

TypeError: 'type' object is not iterable

In [187]:
%%time
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_sparse, y)
logit_test_pred_proba = sgd_logit.predict_proba(X_sparse[400411:])

Wall time: 675 ms


In [192]:
write_to_submission_file(logit_test_pred_proba[:, 1], r'C:\Users\Дмитрий Мошкин\Documents\GitHub\moshkin\MFTI_Yandex_DA_specialisation\6 course\w5_a2.csv')

In [188]:
len(logit_test_pred_proba[:, 1])

82797

## Критерии оценки работы (только для Peer Review в специализации):
- Правильные ли получились размерности матриц в п. 1? (max. 2 балла)
- Правильным ли получилось значения ROC AUC в п. 2? (max. 4 балла)
- Побит ли бенчмарк "sgd_logit_benchmark.csv" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)
- Побит ли бенчмарк "Logit +3 features" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)

## Пути улучшения
На этой неделе дается много времени на соревнование. Не забывайте вносить хорошие идеи, к которым Вы пришли по ходу соревнования, в описание финального проекта (`html`, `pdf` или `ipynb`). Это только в случае, если вы проходите специализацию.
Что можно попробовать:
 - Использовать ранее построенные признаки для улучшения модели (проверить их можно на меньшей выборке по 150 пользователям, отделив одного из пользователей от остальных – это быстрее)
 - Настроить параметры моделей (например, коэффициенты регуляризации)
 - Если позволяют мощности (или хватает терпения), можно попробовать смешивание (блендинг) ответов бустинга и линейной модели. [Вот](http://mlwave.com/kaggle-ensembling-guide/) один из самых известных тьюториалов по смешиванию ответов алгоритмов, также хороша [статья](https://alexanderdyakonov.wordpress.com/2017/03/10/cтекинг-stacking-и-блендинг-blending) Александра Дьяконова
 - Обратите внимание, что в соревновании также даны исходные данные о посещенных веб-страницах Элис и остальными 1557 пользователями (*train.zip*). По этим данным можно сформировать свою обучающую выборку. 

На 6 неделе мы пройдем большой тьюториал по Vowpal Wabbit и попробуем его в деле, на данных соревнования.